In [27]:
# 조회? 하는 최근접 취약시설의 수
n_neighbors = 5

In [28]:
import numpy as np
from sklearn.neighbors import KDTree
from math import radians, sin, cos, sqrt, atan2
import pandas as pd

In [29]:
facility_url = "../서울시 취약시설 데이터 수집/make_file/(최종)_서울시_모든_시설의_위치.csv"
road_url = "./make_file/(경사도)_서울열선_광진도로.csv"

file_encoding = "UTF-8"

In [30]:
facility_data = pd.read_csv(facility_url, encoding = file_encoding)
road_data = pd.read_csv(road_url, encoding = file_encoding)

In [31]:
facility_data.columns

Index(['Unnamed: 0', '시설_구분', '시설_이름', '시설_위치', '위도', '경도'], dtype='object')

In [32]:
road_data.columns

Index(['Unnamed: 0', '도로명', '열선', '도로 종류', '시작점_위도', '시작점_경도', '종료점_위도',
       '종료점_경도', '중앙점_위도', '중앙점_경도', '도로_길이', '행정동', '도로폭', '도로규모', '행정구역',
       '시작점_고도', '종료점_고도', '고도의_차이', '경사각'],
      dtype='object')

In [33]:
try : 
    facility_data = facility_data.drop(columns= 'Unnamed: 0')
    print("삭제 완료")
except KeyError as e :
    print(e)

삭제 완료


In [34]:
try : 
    road_data = road_data.drop(columns= '시작점_고도')
    print("삭제 완료")
except KeyError as e :
    print(e)

삭제 완료


In [35]:
try : 
    road_data = road_data.drop(columns= '종료점_고도')
    print("삭제 완료")
except KeyError as e :
    print(e)

삭제 완료


In [36]:
try : 
    road_data = road_data.drop(columns= 'Unnamed: 0')
    print("삭제 완료")
except KeyError as e :
    print(e)

삭제 완료


In [37]:
# 도로 중앙값 데이터 (위경도)
road_lat_list = road_data['중앙점_위도'].to_list()
road_lon_list = road_data['중앙점_경도'].to_list()

# 취약시설 데이터 (시설명, 위경도) 
facility_lat_list = facility_data["위도"].to_list()
facility_lon_list = facility_data["경도"].to_list()


In [38]:
print(f"road_lat_list 길이 : {len(road_lat_list)}")
print(f"road_lon_list 길이 : {len(road_lon_list)}")
 
print(f"facility_lat_list 길이 : {len(facility_lat_list)}")
print(f"facility_lon_list 길이 : {len(facility_lon_list)}")

road_lat_list 길이 : 7212
road_lon_list 길이 : 7212
facility_lat_list 길이 : 11908
facility_lon_list 길이 : 11908


## 거리를 구하기

In [39]:
def haversine(lat1, lon1, lat2, lon2): 
    R = 6371  # 지구 반경 (km)
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    result = R * c * 1000 
    return result # 미터 단위로 반환

In [40]:
def find_nearest_facilities_with_haversine(road_lats, road_lons, facility_lats, facility_lons):
    global n_neighbors
    # KDTree로 시설 좌표 인덱스를 트리 구조로 저장
    facilities = np.array(list(zip(facility_lats, facility_lons)))
    tree = KDTree(facilities)

    # 각 도로에 대해 최근접 n개의 시설 찾기
    distance_matrix = []
    avg_distances = []
    
    for r_lat, r_lon in zip(road_lats, road_lons):
        road_coords = np.array([[r_lat, r_lon]])
        
        # 최근접 n개 시설의 인덱스 및 거리 계산
        dist, indices = tree.query(road_coords, k=n_neighbors)
        
        # 거리 계산하여 2차원 배열에 저장
        distances = []
        for i in range(n_neighbors):
            dist_val = haversine(r_lat, r_lon, facility_lats[indices[0][i]], facility_lons[indices[0][i]])  # Haversine 거리 계산
            distances.append(dist_val)
        
        distance_matrix.append(distances)
        
        # 평균 거리 계산하여 저장
        avg_distance = np.mean(distances)
        avg_distances.append(avg_distance)
    
    return np.array(distance_matrix), np.array(avg_distances)

In [41]:
# 거리 계산
distance_matrix, avg_distances = find_nearest_facilities_with_haversine(road_lat_list, road_lon_list, facility_lat_list, facility_lon_list)

In [42]:
print(f"수집한 도로_취약시설 거리의 수 : {len(distance_matrix)}")
print(f"수집한 도로_취약시설 거리의 평균 수 : {len(avg_distances)}")

수집한 도로_취약시설 거리의 수 : 7212
수집한 도로_취약시설 거리의 평균 수 : 7212


In [43]:
print(distance_matrix[1])

[ 62.3682751   84.24249386  84.24249386 111.44094049 111.44094049]


In [44]:
distance_matrix = distance_matrix.tolist()

In [45]:
new_distance_matrix = [" ,".join(map(str, row)) for row in distance_matrix]

In [46]:
new_distance_matrix[1]

'62.36827510450082 ,84.24249385836922 ,84.24249385836922 ,111.44094049421805 ,111.44094049421805'

In [47]:
road_data['최근접_시설들_거리'] = new_distance_matrix
road_data['최근접_시설의_평균거리'] = avg_distances

In [48]:
road_data['최근접_시설들_거리'].head()

0    165.17299640951381 ,141.88730282955694 ,165.97...
1    62.36827510450082 ,84.24249385836922 ,84.24249...
2    117.13714873310688 ,122.47106043434799 ,181.43...
3    170.7456882318119 ,207.35031231752598 ,261.112...
4    119.57304016628333 ,132.2920410496931 ,236.865...
Name: 최근접_시설들_거리, dtype: object

In [49]:
print(road_data['최근접_시설의_평균거리'].mean())

170.6477834520203


In [50]:
road_data.columns

Index(['도로명', '열선', '도로 종류', '시작점_위도', '시작점_경도', '종료점_위도', '종료점_경도', '중앙점_위도',
       '중앙점_경도', '도로_길이', '행정동', '도로폭', '도로규모', '행정구역', '고도의_차이', '경사각',
       '최근접_시설들_거리', '최근접_시설의_평균거리'],
      dtype='object')

In [51]:
try :
    save_url = "./make_file/(거리측정)_서울열선_광진도로.csv"
    road_data.to_csv(save_url, encoding=file_encoding, index=False)
    print(f"{save_url} 저장 완료. ")
except OSError as e :
    print(e)

./make_file/(최종)_서울열선_광진도로.csv 저장 완료. 
